In [1]:
import xml.etree.cElementTree as ET
add='D:/Anaconda3/mongodb-json-files/chicago.osm'
tree=ET.ElementTree(file=add)
root=tree.getroot()

'addr:street' is expected to be a combination of 'addr:street:name', 'addr:street:prefix', 'addr:street:suffix' and  'addr:street:type'. First of all, are there any exceptions to this? Before removing the latter four keys we should answer to this. 



In [2]:
# detect cases where 'addr:street' does not exist but some of 'addr:street:name', 
#'addr:street:prefix', 'addr:street:suffix' and  'addr:street:type' exist

for elem in root:
    if elem.tag=='node' or elem.tag=='way':
        street, name, suffix, prefix,type_=False, False, False, False, False
        for child in elem:
            if child.tag=='tag':
                if child.attrib['k']=='addr:street':
                    street=True
                    break
                if child.attrib['k']=='addr:street:name':
                    name=True
                if child.attrib['k']=='addr:street:suffix':
                    suffix=True
                if child.attrib['k']=='addr:street:prefix':
                    prefix=True
                if child.attrib['k']=='addr:street:type':
                    type_=True
        if not street:
            if name or suffix or prefix or type_:
                print(elem.tag, elem.attrib)
                for item in elem:
                    print(item.tag, item.attrib)

node {'changeset': '16929182', 'id': '2382586842', 'lat': '41.778945', 'lon': '-87.66389', 'timestamp': '2013-07-12T18:32:25Z', 'uid': '1237170', 'user': 'Zol87', 'version': '4'}
tag {'k': 'addr:city', 'v': 'Chicago'}
tag {'k': 'addr:housenumber', 'v': '6315'}
tag {'k': 'addr:postcode', 'v': '60636'}
tag {'k': 'addr:street:name', 'v': 'Ashland'}
tag {'k': 'addr:street:prefix', 'v': 'South'}
tag {'k': 'addr:street:type', 'v': 'Avenue'}
tag {'k': 'alt_name', 'v': 'Ashland'}
tag {'k': 'chicag:building_id', 'v': '534249'}
tag {'k': 'chicago_buildingid', 'v': '534249'}
tag {'k': 'name', 'v': 'Ashland/63rd'}
tag {'k': 'operator', 'v': 'Chicago Transit Authority'}
tag {'k': 'public_transport', 'v': 'station'}
tag {'k': 'railway', 'v': 'station'}
tag {'k': 'website', 'v': 'http://www.transitchicago.com/travel_information/station.aspx?StopId=20'}
tag {'k': 'wheelchair', 'v': 'yes'}
tag {'k': 'wikipedia', 'v': 'en:Ashland/63rd %28CTA station%29'}
way {'changeset': '15574076', 'id': '210258350', 

In [4]:
#create sub_elements containing 'addr:street' key for the above exceptional cases
#and remove 'addr:street:name', 'addr:street:prefix', 'addr:street:suffix', 'addr:street:type' for these exceptionals

attrib_dict1={'k':'addr:street', 'v':'South Ashland Avenue'}
attrib_dict2={'k':'addr:street', 'v':'West 14th Place'}

for elem in root:
    if elem.tag=='node' and elem.attrib['id']=='2382586842':
        ET.SubElement(elem,'tag',attrib=attrib_dict1)
        for child in elem:
            if child.tag=='tag':
                if child.attrib['k']=='addr:street:name':
                    elem.remove(child)
                if child.attrib['k']=='addr:street:prefix':
                    elem.remove(child)
                if child.attrib['k']=='addr:street:suffix':
                    elem.remove(child)
                if child.attrib['k']=='addr:street:type':
                    elem.remove(child)
    if elem.tag=='way' and elem.attrib['id']=='210258350':
        ET.SubElement(elem,'tag',attrib=attrib_dict2)
        for child in elem:
            if child.tag=='tag':
                if child.attrib['k']=='addr:street:name':
                    elem.remove(child)
                if child.attrib['k']=='addr:street:prefix':
                    elem.remove(child)
                if child.attrib['k']=='addr:street:suffix':
                    elem.remove(child)
                if child.attrib['k']=='addr:street:type':
                    elem.remove(child)

tree.write("chicago.osm",encoding='UTF-8')  

In [19]:
#we must be sure that removing the keys 'addr:street:name', 'addr:street:prefix', 'addr:street:suffix', 'addr:street:type'
#does not lead to any loss of information

for elem in root:
    if elem.tag=='node':
        street, name, prefix, type_=[False,""], [False,""], [False,""], [False,""]
        for child in elem:
            if child.tag=='tag':
                if child.attrib['k']=='addr:street':
                    street[0]=True
                    street[1]=child.attrib['v'].strip().lower()
                    
                if child.attrib['k']=='addr:street:name':
                    name[0]=True
                    name[1]=child.attrib['v'].strip().lower()
               
                if child.attrib['k']=='addr:street:prefix':
                    prefix[0]=True
                    prefix[1]=child.attrib['v'].strip().lower()
                if child.attrib['k']=='addr:street:type':
                    type_[0]=True
                    type_[1]=child.attrib['v'].strip().lower()
        if  street[0] and name[0]:
            if street[1]!=(prefix[1]+" "+name[1]+" "+type_[1]).strip():
                print(elem.tag, elem.attrib)
                for item in elem:
                    print(item.tag, item.attrib)


In [32]:
# The above cell proves the unnecessity  of 'addr:street:name', 'addr:street:prefix', 'addr:street:suffix', 'addr:street:type'
# delete these keys
for elem in root:
    if elem.tag=='node' or elem.tag=='way':
        for sub_elem in elem:
            if sub_elem.tag=='tag':
                if sub_elem.attrib['k']=='addr:street:name':
                    elem.remove(sub_elem)
                elif sub_elem.attrib['k']=='addr:street:prefix': 
                    elem.remove(sub_elem)
                elif sub_elem.attrib['k']=='addr:street:suffix':
                    elem.remove(sub_elem)
                elif sub_elem.attrib['k']=='addr:street:type':
                    elem.remove(sub_elem)
                
                    
tree.write("chicago.osm",encoding='UTF-8')  

In [38]:
#three modifications
#delete 101 from the addr:street value '101 West Vallette Street'
#delete 255 from the addr:street value '255 North Oakhurst Drive'
#'West Pope John Paul Ii Drive'---->'West Pope John Paul II Drive'                    



for elem in root:
    if elem.tag=='node' or elem.tag=='way':
        for sub_elem in elem:
            if sub_elem.tag=='tag':
                if sub_elem.attrib['k']=='addr:street':
                    if sub_elem.attrib['v']=='101 West Vallette Street':
                        sub_elem.attrib['v']='West Vallette Street'
                    elif sub_elem.attrib['v']=='255 North Oakhurst Drive':
                        sub_elem.attrib['v']='North Oakhurst Drive'
                    elif sub_elem.attrib['v']=='West Pope John Paul Ii Drive':
                        sub_elem.attrib['v']='West Pope John Paul II Drive'
                        
tree.write("chicago.osm",encoding='UTF-8')    
                
        

In [40]:

def highways(street_name):
    if 'hwy' in street_name:
        street_name[street_name.index('hwy')]='Highway'
    elif 'us' in street_name:
        if 'rte.' not in street_name and 'route' not in street_name and 'rte' not in street_name:
            street_name[street_name.index('us')]='US Route'
        else:
            street_name[street_name.index('us')]='US'
            if 'rte.' in street_name:
                street_name[street_name.index('rte.')]='Route'
            if 'rte' in street_name:
                street_name[street_name.index('rte')]='Route'
            if 'route' in street_name:
                street_name[street_name.index('route')]='Route'
    
    else:
        if 'rte.' in street_name:
            street_name[street_name.index('rte.')]='US Route'
        if 'rte' in street_name:
            street_name[street_name.index('rte')]='US Route'
        if 'route' in street_name:
            street_name[street_name.index('route')]='US Route'
        
        
        
        

            
    return street_name

In [41]:
def from_list_to_string(list_of_words):
    str_=""
    for word in list_of_words:
        for part in  word.split(' '):
            if part!='':
                if part.lower()=='us':
                    str_=str_+'US'+' '
                elif part.lower()=='il':
                    str_=str_+'IL'+' '
                elif part.lower()=='and':
                    str_=str_+'and'+' '
                else:
                    str_=str_+part.capitalize().strip()+' '

    return str_.strip()

In [42]:
#updating street names

special_words= ["street","avenue", "boulevard", "drive", "court", "place", "square", "lane", "road",  "trail", "parkway", 
                "commons", "route", "highway", "access", "walk", "way", "market", "circle", "row", 'center',"terrace","park",
                "east", "north", "south", "west", "extension", 'plaza','broadway','path']

abbreviations=["st.", "st", "ave.", "ave","av","av.", "blvd.", "blvd", "dr.", "dr","ct.","ct", "ctr.","ctr", "pl.", "terr.","terr",
               "pl","sq.","sq", "pk.","pk", "ln","ln.", "rd.","rd",  "trl","trl.", "pkwy", "pkwy.", "pwy.","pwy",
               "cmn.", "cmn", "rte.","rte", "hwy.","hwy", "accs.","accs", "wlk.","wlk", "wy.", "wy", 'ter.',"ter",
                "mkt.","mkt", "cir.","cir", "e","e.","n","n.","s.","s","w.","w","ex","ex.",'plz.','plz' ]

directions=["e","e.","n","n.","s.","s","w.","w"]


mapping = { "st": "Street", "st.": "Street", 'rd':'Road', 'rd.':'Road', 'blvd':'Boulevard', 'blvd.':'Boulevard',
           'ave':'Avenue', 'ave.':'Avenue','av':'Avenue','av.':'Avenue','dr':'Drive','dr.':'Drive',
            'pkwy':'Parkway', 'pkwy':'Parkway', 'pwy.':'Parkway','pwy':'Parkway', 'plz.':'Plaza','plz':'Plaza',
           'ct.':'Court', 'ct':'Court', 'ln':'Lane', 'ln.':'Lane', 'trl':'Trail', 'trl.':'Trail', 
           'ter.':'Terrace', 'ter':'Terrace','terr':'Terrace','terr.':'Terrace', 'pl':'Place','pl.':'Place',
           "cmn.":'Commons', "cmn":'Commons', "rte.":'Route',"rte":'Route', "hwy.":'Highway',
           "hwy":'Highway', "accs.":'Access',"accs":'Access', "wlk.":'Walk',"wlk":'Walk', "wy.":'Way', "wy":'Way', 
                "mkt.":'Market',"mkt":'Market', "cir.":'Circle',"cir":'Circle', "e":'East', 'pk':'Park',
           "e.":'East',"n":'North',"n.":'North',"s.":'South',"s":'South',"w.":'West', 'pk.':'Park',
           "w":'West',"ex":'Extension',"ex.":'Extension','ctr.':'Center', 'ctr'
          :'Center'}

exceptions={'Armitage':'Armitage Avenue','Damen':'Damen Avenue','Loomis':'Loomis Street', 'Buren':'Buren Avenue',
             'Sd':'Square', 'Prospect':'Prospect Square',  'Ashland':'Ashland Avenue', 'Glenwood':'Glenwood Avenue',
           'Wabansia':'Wabansia Avenue', 'Grand':'Grand Avenue','Paulina':'Paulina Street', 'Bouleverd':'Boulevard',
           'Belmont':'Belmont Avenue',  'Elston':'Elston Avenue',  'Montrose':'Montrose Avenue', 'Kedzie':'Kedzie Avenue', 
                'Justamere':'Justamere Drive','Milwaukee':'Milwaukee Avenue', 'Shabbona':'Shabbona Street'}
          

def direction_converter(abb):
    if abb[0]=='e':
        return 'East'
    elif abb[0]=='w':
        return 'West'
    elif abb[0]=='s':
        return 'South'
    else:
        return 'North'


def update_street_name(name):
    
    street_name=name.lower().strip().split(" ")
    while street_name.count('')!=0:
        street_name.remove('')
                    
    if street_name[-1].isdigit():
        street_name=highways(street_name)
                        
                        
    else:
        if street_name[-1]  in abbreviations and street_name[-1] not in directions:
            street_name[-1]=mapping[street_name[-1]]
        elif street_name[-1].capitalize() in exceptions.keys():
            street_name[-1]=exceptions[street_name[-1].capitalize()]
        elif street_name[-1] in directions:
            if len(street_name)==3 and  (street_name[0]=='s' or street_name[0]=='s.' or street_name[0]=='south') and (street_name[1]=='ave' or street_name[1]=='ave.' or street_name[1]=='avenue'):
                street_name[-1]=street_name[-1].capitalize()
            else:
                street_name[-1]=direction_converter(street_name[-1])
                    
        else:
            #the last word belongs to the list special words
            street_name[-1]=street_name[-1].capitalize()

                            
                                    
                            
                          
                    
    for count,word in enumerate(street_name[:-1]):
        if word in directions:
            street_name[count]=direction_converter(word)
                     
    return from_list_to_string(street_name)


In [43]:
# update street names in xml file
for elem in root:
    if elem.tag=='node' or elem.tag=='way':
        for sub_elem in elem:
            if sub_elem.tag=='tag':
                if sub_elem.attrib['k']=='addr:street':
                    sub_elem.attrib['v']=update_street_name(sub_elem.attrib['v'])
                    
tree.write("chicago.osm",encoding='UTF-8') 